# Описание проекта

Проект решает задачу выбора оптимального региона для бурения новой скважины в компании «ГлавРосГосНефть». На основании исторических данных трёх регионов (признаки скважин и фактические объёмы) обучается модель прогноза запасов для новых точек. Для каждого региона модель предсказывает объёмы, из которых отбираются скважины с наибольшим ожидаемым результатом. Далее рассчитывается ожидаемая прибыль с учётом цены нефти и затрат. Риски оцениваются методом бутстрапа: строится распределение прибыли, вычисляются среднее значение, доверительный интервал и вероятность убытка. Итогом является выбор региона с максимальной ожидаемой прибылью при приемлемом уровне риска и список рекомендованных скважин.

**План проекта:**

1. Подготовка данных

* Загрузить три набора данных (geo_data_0.csv, geo_data_1.csv, geo_data_2.csv).

* Проверить наличие пропускови, аномалий.

**2. Обучение и проверка модели**

Для каждого региона:

* Разделить данные на обучающую и валидационную выборки (75:25).

* Обучить модель линейной регрессии.

* Сделать предсказания на валидационной выборке.

* Сохранить предсказания и фактические значения.


**3. Подготовка к расчёту прибыли**

Задать исходные параметры:

* Бюджет = 10 млрд ₽,

* Доход с 1 тыс. баррелей = 450 тыс. ₽,

* Количество скважин для разведки = 500,

* Количество лучших скважин для разработки = 200.

* Рассчитать точку безубыточности (минимальный объём сырья на одну скважину).

* Сравнить её со средним запасом по каждому региону.

* Сделать предварительный вывод.

**4. Функция расчёта прибыли**

Реализовать функцию, которая:

* выбирает 200 скважин с наибольшими прогнозами,

* суммирует фактические запасы по этим скважинам,

* считает прибыль.

**5. Анализ рисков и прибыли (Bootstrap)**

* Для каждого региона провести симуляцию (1000 выборок).

Рассчитать:

* среднюю прибыль,

* 95%-й доверительный интервал,

* вероятность убытков.

**6. Итоговый выбор региона**

* Исключить регионы, где вероятность убытков > 2.5%.

* Среди оставшихся выбрать регион с максимальной средней прибылью.


In [1]:
try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats as st
    
    from sklearn.metrics import (
        r2_score,
        mean_squared_error,
        mean_absolute_error
    )
    
    from sklearn.model_selection import (
        train_test_split,
        RandomizedSearchCV,
        GridSearchCV
    )
    
    from sklearn.preprocessing import (
        MinMaxScaler,
        StandardScaler,
        PolynomialFeatures
    )
    
    from sklearn.pipeline import Pipeline
    
    from sklearn.svm import SVR
    
    from sklearn.linear_model import (
        LinearRegression,
        Ridge,
        Lasso
    )
    
    from sklearn.feature_selection import (
        SelectKBest,
        f_regression
    )
except ModuleNotFoundError:

    %pip install -U numpy pandas matplotlib seaborn scikit-learn scipy

    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats as st
    
    from sklearn.metrics import (
        r2_score,
        mean_squared_error,
        mean_absolute_error
    )
    
    from sklearn.model_selection import (
        train_test_split,
        RandomizedSearchCV,
        GridSearchCV
    )
    
    from sklearn.preprocessing import (
        MinMaxScaler,
        StandardScaler,
        PolynomialFeatures
    )
    
    from sklearn.pipeline import Pipeline
    
    from sklearn.svm import SVR
    
    from sklearn.linear_model import (
        LinearRegression,
        Ridge,
        Lasso
    )
    
    from sklearn.feature_selection import (
        SelectKBest,
        f_regression
    )


In [2]:
RANDOM_STATE = 42

In [3]:
def view_metrics(y_test, y_pred):

    print(f"R2 = {round(r2_score(y_test, y_pred),4)}")
    print(f"RMSE = {round(mean_squared_error(y_test,y_pred, squared=False),2)}")
    print(f"MAE = {round(mean_absolute_error(y_test, y_pred),2)}")

# Загрузка данных и предварительный осмотр

In [4]:
try:
    geo_data_0 = pd.read_csv("geo_data_0.csv")
    geo_data_1 = pd.read_csv("geo_data_1.csv")
    geo_data_2 = pd.read_csv("geo_data_2.csv")
except:
    geo_data_0 = pd.read_csv("/datasets/geo_data_0.csv")
    geo_data_1 = pd.read_csv("/datasets/geo_data_1.csv")
    geo_data_2 = pd.read_csv("/datasets/geo_data_2.csv")

In [5]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
geo_data_0.head(5)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [9]:
geo_data_1.head(5)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [10]:
geo_data_2.head(5)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [11]:
geo_data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [12]:
geo_data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [13]:
geo_data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


### Вывод:

Первичный анализ показал, что в данных нет ни пропусков, ни аномальных значений.

# Подготовка данных и обучение линейной модели

In [14]:
# Функция для разбиения данных
def X_y(data):
    X = data.drop(columns="product")
    y = data["product"]
    return X,y

In [15]:
# Функция для разбиения данных на тренировочную и валидационную выборкия
def t_t_s(data):
    X, y  = X_y(data)
    X_train_geo, X_valid_geo, y_train_geo, y_valid_geo = train_test_split(
        X,
        y,
        test_size=0.25,
        random_state = RANDOM_STATE
        )
    geo_index = pd.Series(X_valid_geo.id)
    X_train_geo = X_train_geo.drop(columns="id")
    X_valid_geo = X_valid_geo.drop(columns="id")
    return X_train_geo, X_valid_geo, y_train_geo, y_valid_geo, geo_index

In [16]:
# Функция объединения данных
def concat_data(geo_index, y_valid_geo,product_predict):
    oil_reserves = pd.concat([
        geo_index.reset_index(drop=True),
        y_valid_geo.reset_index(drop=True),
        product_predict
    ],axis=1)
    
    oil_reserves = oil_reserves.rename( columns={0:"predict_product"})
    return oil_reserves
    

### Предсказания объема запасов нефти в 1 регионе

In [17]:
X_train_geo_0, X_valid_geo_0, y_train_geo_0, y_valid_geo_0, geo_index_0 = t_t_s(geo_data_0)


In [18]:
# Пайплайн для подготовки данных и обучения моделей
pipeline_final = Pipeline(
    [
        ("num", StandardScaler()),
        ("polynomial_features", PolynomialFeatures(degree=2, include_bias=False)),
        ("selector", SelectKBest(f_regression)),
        ("model", Ridge(alpha=1.0) )
    ]
)

In [19]:
param_grid = [
    {
        "model__alpha": np.arange(1.0,5.0,0.5),
        "selector__k": range(2,len(X_train_geo_0.columns)+1),
        "num": [StandardScaler(), MinMaxScaler()] 
    },
    {
        "model": [Lasso()],
        "model__alpha": np.arange(1.0,5.0,0.5),
        "selector__k": range(2,len(X_train_geo_0.columns)+1),
        "num": [StandardScaler(), MinMaxScaler()] 
    },

    {
        "model": [LinearRegression()],
        "selector__k": range(2,len(X_train_geo_0.columns)+1),
        "num": [StandardScaler(), MinMaxScaler()] 
    }
]


gs_0 = GridSearchCV(
    pipeline_final,
    param_grid,
    n_jobs=-1,
    scoring = "r2",
    cv=5,
)

In [20]:
gs_0.fit(X_train_geo_0, y_train_geo_0)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('num', StandardScaler()),
                                       ('polynomial_features',
                                        PolynomialFeatures(include_bias=False)),
                                       ('selector',
                                        SelectKBest(score_func=<function f_regression at 0x7fd4b7bdc0d0>)),
                                       ('model', Ridge())]),
             n_jobs=-1,
             param_grid=[{'model__alpha': array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5]),
                          'num': [StandardScaler(), MinMaxScaler()],
                          'selector__k': range(2, 4)},
                         {'model': [Lasso()],
                          'model__alpha': array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5]),
                          'num': [StandardScaler(), MinMaxScaler()],
                          'selector__k': range(2, 4)},
                         {'model': [LinearRegressi

In [21]:
gs_0.best_score_

0.27646863565297514

In [22]:
product_predict_0 = pd.Series(gs_0.predict(X_valid_geo_0))

In [23]:
view_metrics(y_valid_geo_0, product_predict_0)

R2 = 0.2728
RMSE = 37.76
MAE = 31.05


* Сохраним предсказания и правильные ответы

In [24]:
oil_reserves_0 = concat_data(geo_index_0, y_valid_geo_0, product_predict_0)

In [25]:
oil_reserves_0.head(5)

,id,product,predict_product
0,7C6bA,122.073350,101.900490
1,dVxsn,48.738540,78.218695
2,1Gm3l,131.338088,115.265745
3,Nzg1t,88.327757,105.618070
4,Z7b4F,36.959266,97.979951


* Объединение данные произошло успешно

### Предсказания объема запасов нефти в 2 регионе

In [26]:
X_train_geo_1, X_valid_geo_1, y_train_geo_1, y_valid_geo_1, geo_index_1 = t_t_s(geo_data_1)

In [27]:
gs_1 = GridSearchCV(
    pipeline_final,
    param_grid,
    n_jobs=-1,
    scoring = "r2",
    cv=5,
)

In [28]:
gs_1.fit(X_train_geo_1, y_train_geo_1)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('num', StandardScaler()),
                                       ('polynomial_features',
                                        PolynomialFeatures(include_bias=False)),
                                       ('selector',
                                        SelectKBest(score_func=<function f_regression at 0x7fd4b7bdc0d0>)),
                                       ('model', Ridge())]),
             n_jobs=-1,
             param_grid=[{'model__alpha': array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5]),
                          'num': [StandardScaler(), MinMaxScaler()],
                          'selector__k': range(2, 4)},
                         {'model': [Lasso()],
                          'model__alpha': array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5]),
                          'num': [StandardScaler(), MinMaxScaler()],
                          'selector__k': range(2, 4)},
                         {'model': [LinearRegressi

In [29]:
gs_1.best_score_

0.9996209513424705

In [30]:
product_predict_1 = pd.Series(gs_1.predict(X_valid_geo_1))

In [31]:
view_metrics(y_valid_geo_1, product_predict_1)

R2 = 0.9996
RMSE = 0.89
MAE = 0.72


In [32]:
oil_reserves_1 = concat_data(geo_index_1, y_valid_geo_1, product_predict_1)

In [33]:
oil_reserves_1.head(5)

,id,product,predict_product
0,fYVDK,0.000000,0.844738
1,3SQcq,53.906522,52.921612
2,hQ1gJ,134.766305,135.110385
3,Y4vux,107.813044,109.494863
4,H1NUw,0.000000,-0.047292


### Предсказания объема запасов нефти в 3 регионе

In [34]:
X_train_geo_2, X_valid_geo_2, y_train_geo_2, y_valid_geo_2, geo_index_2 = t_t_s(geo_data_2)

In [35]:
gs_2 = GridSearchCV(
    pipeline_final,
    param_grid,
    n_jobs=-1,
    scoring = "r2",
    cv=5,
)

In [36]:
gs_2.fit(X_train_geo_2, y_train_geo_2)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('num', StandardScaler()),
                                       ('polynomial_features',
                                        PolynomialFeatures(include_bias=False)),
                                       ('selector',
                                        SelectKBest(score_func=<function f_regression at 0x7fd4b7bdc0d0>)),
                                       ('model', Ridge())]),
             n_jobs=-1,
             param_grid=[{'model__alpha': array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5]),
                          'num': [StandardScaler(), MinMaxScaler()],
                          'selector__k': range(2, 4)},
                         {'model': [Lasso()],
                          'model__alpha': array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5]),
                          'num': [StandardScaler(), MinMaxScaler()],
                          'selector__k': range(2, 4)},
                         {'model': [LinearRegressi

In [37]:
gs_2.best_score_

0.24999301659031628

In [38]:
product_predict_2 = pd.Series(gs_2.predict(X_valid_geo_2))

In [39]:
view_metrics(y_valid_geo_2, gs_2.predict(X_valid_geo_2))

R2 = 0.2463
RMSE = 38.88
MAE = 32.03


In [40]:
oil_reserves_2 = concat_data(geo_index_2, y_valid_geo_2, product_predict_2)

In [41]:
oil_reserves_2.head(5)

,id,product,predict_product
0,sPjbT,117.441301,100.518293
1,1u4uo,47.841249,93.454004
2,kZ6HO,45.883483,53.938540
3,gCZGK,139.014608,110.440447
4,rqJHx,84.004276,78.574162


# Подготовка к расчета прибыли

In [42]:
BUDGET = 10e9
REVENUE = 450e3
N = 200
N_RESEARCH = 500
mean_oil_reserves_0 = oil_reserves_0["predict_product"].mean()
mean_oil_reserves_1 = oil_reserves_1["predict_product"].mean()
mean_oil_reserves_2 = oil_reserves_2["predict_product"].mean()

*  Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. 

In [43]:
(BUDGET // REVENUE + 1)/N

111.115

* Точка безубыточности — 112 тыс. баррелей на скважину.

In [44]:
mean_oil_reserves_0

92.39880986209452

In [45]:
mean_oil_reserves_1

68.7128780391376

In [46]:
mean_oil_reserves_2

94.7613018551857

### Вывод:

Средние запасы во всех трёх регионах ниже точки безубыточности (111.11 тыс. баррелей). Особенно сильно отстаёт регион 1, поэтому он выглядит наименее перспективным. Регионы 0 и 2 показывают результаты ближе к порогу, но всё равно немного не дотягивают. Однако это лишь предварительный анализ: расчёт вёлся по среднему значению на всей валидационной выборке. В дальнейшем мы будем рассматривать только 200 скважин с наибольшими прогнозируемыми запасами, и средний объём может оказаться выше.

#  Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [58]:
# Это функция, которая подсчитывает прибыль
# Принимает на вход объект Series(500 истинных значений и 500 предсказанных значений)
def profit_calculation(target_subsample, predict_subsample, N=200, REVENUE=450e3):
    
    # Обнуляем индексы, чтобы не произошло дублирование чисел при срезе
    target_subsample = target_subsample.reset_index(drop=True)
    predict_subsample = predict_subsample.reset_index(drop=True)
    
    # Сортируем предсказанные значения выбираем значения и выбираем 200 лучших, берем их индексы
    target_subsample = target_subsample[predict_subsample.sort_values(ascending = False)[:N].index]
    
    # Считаем прибыль
    total_profit = round(sum(target_subsample * REVENUE) - BUDGET,2)
    return total_profit

In [59]:
profit_calculation(oil_reserves_0["product"], oil_reserves_0["predict_product"])

3359141114.46

In [60]:
profit_calculation(oil_reserves_1["product"], oil_reserves_1["predict_product"])

2415086696.68

In [61]:
profit_calculation(oil_reserves_2["product"], oil_reserves_2["predict_product"])

2780026498.76

### Вывод: 

Во всех регионах разработка скважин обеспечивает положительный результат — суммарный объём добычи перекрывает затраты, то есть проект в каждом регионе выходит в плюс.

# Подсчет рисков и прибыли для каждого региона

* Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли

In [62]:
state = np.random.RandomState(12345)

def bootstrap(data): # data состоит из predict_product и product(этот датафрейм создается в функции "concat_data")
    values = []
    
    for i in range(1000):
        #Здесь мы семплируем данные, получаем 500 случайных предсказанных значений.
        predict_subsample = data["predict_product"].sample(n = N_RESEARCH, replace=True, random_state = state)
        
        #Здесь мы получаем по индексам предказанных значений те же 500 значений, но только уже истинных значений
        target_subsample = data["product"][predict_subsample.index]
        
        #Добавляем в переменную values значение прибыли. Передаем в функцию "fit_calculation" те же 500 истинных значений и предсказанных значений,
        values.append(profit_calculation(target_subsample, predict_subsample))

    return values


        

In [63]:
# Средняя прибыль
def mean_profit(data):
    return round(data.mean(),2)

# Доверительный интервал
def conf_interval(data):
    lower = data.quantile(0.025)
    upper = data.quantile(0.975)

    return lower, upper

# Риск убытков
def risk_of_loss(data):
    share = len(data[data<0])/len(data)*100
    return round(share,2)


In [64]:
def general_information_about_region(oil_reserves, budget=10e9):
    oil_reserves_bootstrap = pd.Series(bootstrap(oil_reserves))

    print(f"Средняя прибыль = {mean_profit(oil_reserves_bootstrap)}")
    print(f"95%-й доверительный интервал = {conf_interval(oil_reserves_bootstrap)}")

    if risk_of_loss(oil_reserves_bootstrap)>2.5:
        print(f"В этом регионе риск убытков больше 2.5%. 2.5% квартиль = {risk_of_loss(oil_reserves_bootstrap)}. Этот регион нам не подходит")
    else:
        print(f"В этом регионе риск убытков меньше 2.5%. Этот регион нам подходит. 2.5% квартиль = {risk_of_loss(oil_reserves_bootstrap)}")
        


In [65]:
general_information_about_region(oil_reserves_0)

Средняя прибыль = 406270075.82
95%-й доверительный интервал = (-117742136.49274999, 911737050.7469999)
В этом регионе риск убытков больше 2.5%. 2.5% квартиль = 6.7. Этот регион нам не подходит


In [66]:
general_information_about_region(oil_reserves_1)

Средняя прибыль = 441504277.59
95%-й доверительный интервал = (35728489.27925002, 828006639.004)
В этом регионе риск убытков меньше 2.5%. Этот регион нам подходит. 2.5% квартиль = 1.6


In [67]:
general_information_about_region(oil_reserves_2)

Средняя прибыль = 582026614.77
95%-й доверительный интервал = (57259053.116000004, 1076830950.0979998)
В этом регионе риск убытков меньше 2.5%. Этот регион нам подходит. 2.5% квартиль = 1.8


### Вывод:

Мы применили технику Bootstrap, чтобы оценить распределение прибыли по регионам. Для каждого региона были рассчитаны:

* средняя прибыль,

* 95%-й доверительный интервал,

* вероятность убытков.

Из трёх регионов только два показали риск убытков ниже 2.5%. Среди них для выбора ключевым критерием стала средняя прибыль. Максимальное значение средней прибыли оказалось у региона № 3, поэтому именно его рекомендуется выбрать для разработки.

# Вывод:

Предварительный анализ показал, что средние запасы во всех регионах не превышают точку безубыточности, однако при детальном рассмотрении наиболее перспективных скважин ситуация улучшается. Проведённый с помощью Bootstrap расчёт прибыли показал, что только во втором и третьем регионах положительный результат разработки скважин: суммарные объёмы добычи перекрывают затраты. Таким образом, ключевым критерием выбора стал показатель средней прибыли. Наибольшая средняя прибыль была получена в третьем регионе, поэтому именно его рекомендуется выбрать для разработки.